# VTHacks 8: Deep Reinforcement Learning
### GitHub Repo: https://github.com/AndrewAF1/vthacks-rl-workshop
### Presentors: Andrew Farabow (aafarabow@vt.edu), Patrick Riley (rileyp@vt.edu)

## What is reinforcement learning?

### Short Answer
Using intelligent trial and error to solve problems

### Long Answer 
A reinforcement learning problem consists of an agent which interacts with an environment in discrete timesteps. At each timestep the agent is given an observation $s$, takes an action a based on policy $\pi$, and receives a reward $r$. The policy, which defines an agent’s behavior, is a function that maps the state to a probability distribution over the (discrete or continuous) set of possible actions. The defining characteristic of deep reinforcement learning is that the policy is represented by a neural network, which is generally trained via some variant of stochastic gradient descent to maximize expected reward. In order to make a challenge learnable via RL, one must define an observation and action space. These are vectors that define the shape and range of possible values that can compose the observation and action. The observation space consists of the features of the game we let the agent learn to make decisions from, and the action space is determined by what parts of the environment we decide to have the agent control. 

### Translation
RL involves giving an "agent" a goal (along with a way of measuring that goal) and allowing to figure out how to reach it on its own. It does this by playing the game repeatedly and trying to improve its score by combining exploration (trying new things) with exploitation (doing what it has learned works well).


## What does the "deep" part mean?
There are many ways to solve RL problems. "Deep" means that a neural network is used to represent the "brain" of the algorithm - it encodes the method by which the agent makes decisions and allows that method to be updated as new data becomes available.

## What we are going to do today
* solve a classic reinforcement learning problem, Cartpole
* explore a basic RL algorithm, Deep Q-Learning
* tune various parameters and see how that affects learning

## Our tools
* Python (NOTE: ADD LINKS LATER)
* PyTorch
* OpenAI Gym

![Cartpole](https://i.redd.it/sqjzj2cgnpt21.gif)

## Lets start the code!

Below, we are going to set up a few parameters that will be used in training. Some won't make sense yet.
They are named after the Greek letters used in the equations from the DQN paper. The person who wrote this code is cruel.

In [ ]:
import gym
import random

env = gym.make('CartPole-v0') # creates an OpenAI Gym environment, an object you interact with to step through the game
epsilon = .01                 # controls the exploration vs exploitation balance
gamma = .99                   # reward discount factor
tau = .995                    # controls how quickly we update the target network
random.seed(666)              # seed the random number generator for reproducibility
batch_size = 128              # how much to sample from the replay buffer at a time
max_ep = 500                  # number of games to play

The code below creates a neural network in PyTorch. The `nn.Linear...` lines represent fully connected layers and the `nn.ReLU..` lines are activation functions, which allow the network to represent non-linear functions. The `forward` function gets called when you want to get the output of the model for some data (called a forward pass).

In [ ]:
import torch
import torch.nn as nn

class Q(nn.Module):
    def __init__(self,env):
        super(Q, self).__init__()

        self.main = nn.Sequential(
            nn.Linear(env.observation_space.shape[0], 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, env.action_space.n)
        )

    def forward(self, s):
        return self.main(torch.FloatTensor(s))

This is the replay buffer, which stores the states, actions, rewards, next state, and masks (more on those later) for every frame of the game. We will use this data to update the neural network later.

In [ ]:
import random
from collections import deque

class ReplayBuffer():
    def __init__(self, size):
        self.buffer = deque(maxlen=int(size))
        self.maxSize = size
        self.len = 0

    def sample(self, count):
        count = min(count, self.len)
        batch = random.sample(self.buffer, count)

        s_arr = torch.FloatTensor(np.array([arr[0] for arr in batch]))
        a_arr = torch.FloatTensor(np.array([arr[1] for arr in batch]))
        r_arr = torch.FloatTensor(np.array([arr[2] for arr in batch]))
        s2_arr = torch.FloatTensor(np.array([arr[3] for arr in batch]))
        m_arr = torch.FloatTensor(np.array([arr[4] for arr in batch]))

        return s_arr, a_arr.unsqueeze(1), r_arr.unsqueeze(1), s2_arr, m_arr.unsqueeze(1)

    def len(self):
        return self.len

    def store(self, s, a, r, s2, d):
        def fix(x):
            if not isinstance(x, np.ndarray): return np.array(x)
            else: return x

        data = [s, np.array(a,dtype=np.float64), r, s2, 1 - d]
        transition = tuple(fix(x) for x in data)
        self.len = min(self.len + 1, self.maxSize)
        self.buffer.append(transition)

Some imports

In [ ]:
import numpy as np
import torch.nn.functional as F
from copy import deepcopy

This function explores the environment for the specified number of timesteps to improve the performance of the DQN. Gets run at the begining of training


In [ ]:
def explore(timestep):
    ts = 0
    while ts < timestep:
        s = env.reset()
        while True:
            a = env.action_space.sample()
            s2, r, done, _ = env.step(int(a))
            rb.store(s, a, r, s2, done)
            ts += 1
            if done:
                break
            else:
                s = s2

This code updates the Q by sampling past states from the buffer, comparing the network's estimate of a state's potential future rewards versus the actual future rewards.

In [ ]:
def update():
    s, a, r, s2, m = rb.sample(batch_size) # get a batch of states, actions, reward, next states and
                                           # masks (1 for game over, else zero) from the replay buffer

    with torch.no_grad(): # don't track gradients when you pass through the target network
        max_next_q, _ = q_target(s2).max(dim=1, keepdim=True) # get the next state value from the target net
        
        # sum rewards with discount penalty to avoid infinite time horizons
        # reward + "doneness" * discount factor * anticipated future reward as predicted by (target) Q network
        #y = ## FILL IN CODE HERE
    
    q_estimates = torch.gather(q(s), 1, a.long())
    
    # calulcate the MSE loss between the q_estimates and y
    #loss = ## FILL IN CODE HERE

    # update Q network weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # update target Q networked with weighted avereging
    for param, target_param in zip(q.parameters(), q_target.parameters()):
        target_param.data = target_param.data*tau + param.data*(1-tau)


This is the main body of our algorithm, where we use all the bits we defined above to step through the enviornment and learn from experience.

In [ ]:
# init the Q network and target network, a copy that lags behind to smooth training
q = Q(env)
q_target = deepcopy(q)

# init the optimizer, which uses the loss to update the weights of the network
optimizer = torch.optim.Adam(q.parameters(), lr=1e-3)

# init the replay buffer and do some exploration to fill it
rb = ReplayBuffer(1e6)
explore(10000) 

# training loop
ep = 0
while ep < max_ep: # loop through some number of episodes, aka complete games
    s = env.reset() # reset the environment at the state of the game
    ep_r = 0
    while True: # loop through a game frame by frame
        with torch.no_grad():
            # epsilon greedy exploration
            if random.random() < epsilon: # some portion of the time
                # pick a random action to aid in exploration
                #a = # FILL IN CODE HERE
                pass
            else: # the rest of the time
                # get the action recomended by the network
                #a = # FILL IN CODE HERE
                pass 
        
        # take a step in the environment and get the resulting next state, reward, and done boolean
        # FILL IN CODE HERE
        
        rb.store(s, a, r, s2, done) # store in the replay buffer
        ep_r += r # update episode reward

        
        if done: # if a ga,e endsbreak the loop and begin again, 
            if ep % 10 == 0:
                print(f"Episode {ep} Reward: {ep_r}")
            ep += 1
            break
        else: # otherwise continue
            s = s2

        update() # compute loss and update the network

## Congrats, thats deep Q learning

## Next steps for the curious
* read the research paper and dig into the math: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
* check out other algorithms like Policy Gradients and see how they compare
* read more about PyTroch
* read my [blog posts](https://computable.ai/category/fundamentals.html) (shameless-self promotion) 
* read Sutton and Barto's [Reinforcement Learning, second edition: An Introduction](https://www.amazon.com/Reinforcement-Learning-Introduction-Adaptive-Computation/dp/0262039249/ref=pd_all_pref_1/146-5524833-7295228?_encoding=UTF8&pd_rd_i=0262039249&pd_rd_r=9ab2599f-5305-49ee-b05e-6c09ab9771d6&pd_rd_w=StfTr&pd_rd_wg=Oa0y3&pf_rd_p=e6474b7e-8fb6-4ee2-b5d6-a1da55185fe6&pf_rd_r=CRDR50SPECKZ3PCCJ2NQ&psc=1&refRID=CRDR50SPECKZ3PCCJ2NQ) (for the mathematically inclined)